<a href="https://colab.research.google.com/github/FrodoBaggins87/Machine_Learning/blob/main/Custom_Datasets_(going_modular).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Getting data

In [ ]:
import requests
import zipfile
from pathlib import Path
import os
#setup path to data folder
data_path= Path("data/")
image_path=data_path/"food_stuff"
#check if image folder exists or not, if not prepare it
if image_path.is_dir():
  print(f"{image_path} directory exists")
else:
  print(f"Didnt find {image_path}, creating...")
  image_path.mkdir(parents=True, exist_ok=True)
  #the datset that will be used is a formatted dataset being taken from a github file, in general, wont get such formatted data
  #download pizza, steak, sushi data in zip file
  with open(data_path/"food_stuff.zip","wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading data...")
    f.write(request.content)
  #unzip data
  with zipfile.ZipFile(data_path/"food_stuff.zip","r") as zip_ref:
    print("Unzipping food_stuff file...")
    zip_ref.extractall(image_path)


data/food_stuff directory exists


Setting up data in dataloaders


In [ ]:
%%writefile data_setup.py
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

NUM_WORKERS= os.cpu_count()
def create_dataloaders(
    train_dir: str,
    test_dir:str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size:int,
    num_workers: int=NUM_WORKERS
):

  training_data=datasets.ImageFolder(root=train_dir, transform=train_transform)
  testing_data=datasets.ImageFolder(root=test_dir, transform=test_transform)
  class_names=training_data.classes
  train_dataloader=DataLoader(dataset=training_data,
                              batch_size=batch_size,#sample per dataloader
                              num_workers=num_workers,
                              shuffle=True,
                              pin_memory= True)
  test_dataloader=DataLoader(dataset=testing_data,
                            batch_size=batch_size,
                            num_workers=num_workers,
                            shuffle=False,
                            pin_memory= True)
  return train_dataloader, test_dataloader, class_names


Overwriting data_setup.py


In [ ]:
from torchvision import transforms
train_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31), #31 is the most intense
    transforms.ToTensor()#using this gets all values between 0 & 1
])
#not putting TrivialAugmentWide transform in test transforms
test_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()#using this gets all values between 0 & 1
])


In [ ]:
import data_setup
train_dataloader, test_dataloader, class_names= data_setup.create_dataloaders(train_dir= image_path/'train', test_dir= image_path/'test', train_transform=train_transforms, test_transform=test_transforms, batch_size=32)

In [ ]:
train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x785975afff10>,
 ['pizza', 'steak', 'sushi'])

Writing file to build TinyVGG model

In [ ]:
%%writefile model_builder.py
#here RGB images instead of grayscale so in_channels=3
import torch
from torch import nn
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape:int) -> None:
    super().__init__()
    self.conv_block_1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1
                  ),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1
                  ),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2=nn.Sequential(
        nn.Conv2d(hidden_units,hidden_units,kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units,hidden_units,kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)#stride = kernel size by default
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16,
                  out_features=output_shape)
    )
  def forward(self,x:torch.Tensor):
    x=self.classifier(self.conv_block_2(self.conv_block_1(x)))
    return x

Overwriting model_builder.py


In [ ]:
#can import tinyvgg using following code
import model_builder
import torch
device= 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(43)
model_0=model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)
model_0

TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=2560, out_features=3, bias=True)
  )
)

In [ ]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary
summary(model_0,[1,3,64,64])

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 32, 32]           --
│    └─Conv2d: 2-1                       [1, 10, 64, 64]           280
│    └─ReLU: 2-2                         [1, 10, 64, 64]           --
│    └─Conv2d: 2-3                       [1, 10, 64, 64]           910
│    └─ReLU: 2-4                         [1, 10, 64, 64]           --
│    └─MaxPool2d: 2-5                    [1, 10, 32, 32]           --
├─Sequential: 1-2                        [1, 10, 16, 16]           --
│    └─Conv2d: 2-6                       [1, 10, 32, 32]           910
│    └─ReLU: 2-7                         [1, 10, 32, 32]           --
│    └─Conv2d: 2-8                       [1, 10, 32, 32]           910
│    └─ReLU: 2-9                         [1, 10, 32, 32]           --
│    └─MaxPool2d: 2-10                   [1, 10, 16, 16]           --
├─Sequentia

Writing a file engine.py which contains the train_step() and test_step() functions and the train() function which calls both of these functions. Calling it engine as it is the engine of training our model

In [ ]:
%%writefile engine.py
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
              device: torch.device) -> Tuple[float, float]:
  #putting in training mode
  model.train()
  #setup training loss and training accuracy
  train_loss,train_acc=0,0

  for batch,(x,y) in enumerate(dataloader):
    #send data to target device
    x,y=x.to(device),y.to(device)
    #forward pass
    y_pred=model(x)
    #calculate and accumulate losses
    loss=loss_fn(y_pred,y)
    train_loss+=loss.item()
    #optimizer zero grad
    optimizer.zero_grad()
    #loss backward
    loss.backward()
    #optimizer step
    optimizer.step()

    #calculate and accumulate accuracy metric for all batches
    y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    train_acc+=(y_pred_class==y).sum().item()/len(y_pred)

  #getting average loss and accuracy for each batch
  train_loss/=len(dataloader)
  train_acc/=len(dataloader)
  return train_loss, train_acc

def test_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
              device: torch.device) -> Tuple[float,float]:
  #putting in eval mode
  model.eval()
  #setup test loss and test accuracy
  test_loss,test_acc=0,0
  #turn on inference context manager
  with torch.inference_mode():
    #loop through dataloader batches
    for batch,(x,y) in enumerate(dataloader):
      #send data to target device
      x,y=x.to(device),y.to(device)
      #forward pass
      test_pred_logits=model(x)
      #calculate and accumulate loss
      loss=loss_fn(test_pred_logits,y)
      test_loss+=loss.item()
      #calculate and accumulate accuracy
      test_pred_labels=torch.argmax(torch.softmax(test_pred_logits,dim=1),dim=1)
      test_acc+=(test_pred_labels==y).sum().item()/len(test_pred_labels)#can probably also use len(test_pred), not sure both should work i think
  #getting average loss and accuracy for each batch
  test_acc/=len(dataloader)
  test_loss/=len(dataloader)
  return test_loss, test_acc

#defining functions and various required parameters
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs: int,
        device: torch.device) -> Dict[str, list]:
  #create empty results dictionary
  results={"train_loss":[],
           "test_loss":[],
           "train_acc":[],
           "test_acc":[]}
  #looping through train_step() and test_step()
  for epoch in tqdm(range(epochs)):
    train_loss,train_acc=train_step(model=model,
                                    dataloader=train_dataloader,
                                    loss_fn=loss_fn,
                                    optimizer=optimizer,
                                    device=device)
    test_loss, test_acc=test_step(model=model,
                                  dataloader=test_dataloader,
                                  loss_fn=loss_fn,
                                  device=device)
  #print whats happening
    print(
        f"Epoch:{epoch+1}|"
        f"Train Loss:{train_loss:.4f}|"
        f"Training Accuracy: {train_acc:.4f}|"
        f"Test Loss: {test_loss:.4f}|"
        f"Test Accuracy: {test_acc:.4f}"
    )
    #updating result dictionary
    results["train_loss"].append(train_loss)
    results["test_loss"].append(test_loss)
    results["train_acc"].append(train_acc)
    results["test_acc"].append(test_acc)
  return results



Overwriting engine.py


In [ ]:
from torchvision import datasets,transforms
testing_data=datasets.ImageFolder(root=image_path/'test', transform=test_transforms)
testing_data[0][0].shape

torch.Size([3, 64, 64])

In [ ]:
#importing engine.py
import engine
from torch import nn
results=engine.train(model=model_0,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=torch.optim.Adam(params=model_0.parameters(), lr=0.001),
                    loss_fn=nn.CrossEntropyLoss(),
                    epochs=5,
                    device=device)

  0%|          | 0/5 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch:1|Train Loss:1.1184|Training Accuracy: 0.2773|Test Loss: 1.1066|Test Accuracy: 0.2604
Epoch:2|Train Loss:1.1057|Training Accuracy: 0.2852|Test Loss: 1.1378|Test Accuracy: 0.1979
Epoch:3|Train Loss:1.0971|Training Accuracy: 0.2930|Test Loss: 1.1205|Test Accuracy: 0.1875
Epoch:4|Train Loss:1.0981|Training Accuracy: 0.2695|Test Loss: 1.1103|Test Accuracy: 0.2604
Epoch:5|Train Loss:1.1115|Training Accuracy: 0.3086|Test Loss: 1.1169|Test Accuracy: 0.2708


Write file to save model (common practice to store helper functions as utils.py)

In [ ]:
%%writefile utils.py
import torch
from pathlib import Path
def save_model(model:torch.nn.Module,
               target_dir: str,
               model_name:str):
  #creating target directory
  target_dir_path=Path(target_dir)
  target_dir_parh.mkdir(parent=True,
                        exist_ok=True)
  #creating model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path/model_name

  #save the model state_dict
  print(f"Saving model to:{model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [ ]:
import utils
#save model to file
save_model(model=model_0,
           target_dir='models',
           model_name='model_0(going_modular)')

Write 1 script to use all above scripts to train, evaluate and save the model

In [ ]:
%%writefile train.py
#HAVE TO USE ARGPARSE SO THAT THE HYPERPARAMETERS CAN BE SPECIFIED WHILE RUNNING THIS SCRIPT
import os
import torch
import data_setup, engine, model_builder, utils

from torchvision import transforms
#setup hyperparameters
NUM_EPOCHS=5
BATCH_SIZE=32
HIDDEN_UNITS=10
LEARNING_RATE=0.001

#setup directories
train_dir="data/pizza_steak_sushi/train"
test_dir="data/pizza_steak_sushi/test"

#setup target device
device= 'cuda' if torch.cuda.is_available() else 'cpu'

#create transforms
train_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31), #31 is the most intense
    transforms.ToTensor()#using this gets all values between 0 & 1
])
#not putting TrivialAugmentWide transform in test transforms
test_transforms=transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor()#using this gets all values between 0 & 1
])
train_dataloader, test_dataloader, class_names= data_setup.create_dataloaders(train_dir= train_dir,
                                                                              test_dir= test_dir,
                                                                              train_transform=train_transforms,
                                                                              test_transform=test_transforms,
                                                                              batch_size=BATCH_SIZE)
#build model using model_builder
model_0=model_builder.TinyVGG(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)
optimizer=torch.optim.Adam(params=model_0.parameters(), lr=LEARNING_RATE)
loss_fn=nn.CrossEntropyLoss()

#start training from engine.py
results=engine.train(model=model_0,
                    train_dataloader=train_dataloader,
                    test_dataloader=test_dataloader,
                    optimizer=optimizer,
                    loss_fn=loss_fn,
                    epochs=NUM_EPOCHS,
                    device=device)
#save model to file
save_model(model=model_0,
           target_dir='models',
           model_name='model_0(going_modular)')